<a href="https://colab.research.google.com/github/Zoro114/machine_learning/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Dec 28 13:40:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'2020UME4061  (1).pdf'
'2020UME4061 (1).pdf'
'2020UME4061  (2).pdf'
'2020UME4061 (2).pdf'
'2020UME4061 (3).pdf'
'2020UME4061 (4).pdf'
'2020UME4061 (5).pdf'
'2020UME4061 (6).pdf'
'2020UME4061 (7).pdf'
'2020UME4061 (8).pdf'
'2020UME4061 (9).pdf'
 2020UME4061.jpg
'2020UME4061 .pdf'
 2020UME4061_.pdf
 2020UME4061.pdf
 2020UME4061_prashant.docx
 2020UME4061_PRASHANT_PRACTICAL.pdf
'2nd semester registration form.pdf'
'3rd sem registration.pdf'
'Abdul Bari ds algo'
 acknowledgement.pdf
 architecture.gdoc
'ares projects'
 Certificates
 Citations.gsheet
 Classroom
'Coding Ninjas C++ with Data Structures and Algorithms'
'Colab Notebooks'
'Copy of Untitled Diagram.drawio'
'C udemy'
'[DesireCourse.Net] Udemy - The Python Mega Course Build 10 Real World Applications'
'Engineering Mechanics by S. S. Bhavikatti, K. G. Rajashekarappa (z-lib.org).pdf'
'ERC Rover Labelled Science-Box.png'
'Etc documents'
'GSOC poster.png'
 IMG-20210221-WA0006.jpg
 IMG_20210908_192342_932.jpg
'

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 6.00 MiB/s, done.
Resolving deltas: 100% (10403/10403), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov4"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "leafminer" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-28 13:42:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  16.6MB/s    in 10s     

2022-12-28 13:42:15 (15.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/10_VID20220622145715.jpg  
  inflating: data/obj/10_VID20220622145715.txt  
  inflating: data/obj/101_VID20220624172411_WgUbrLb.jpg  
  inflating: data/obj/101_VID20220624172411_WgUbrLb.txt  
  inflating: data/obj/102_VID20220624172411_WgUbrLb.jpg  
  inflating: data/obj/102_VID20220624172411_WgUbrLb.txt  
  inflating: data/obj/108_VID20220624172659.jpg  
  inflating: data/obj/108_VID20220624172659.txt  
  inflating: data/obj/11_VID20220622145715.jpg  
  inflating: data/obj/11_VID20220622145715.txt  
  inflating: data/obj/110_VID20220624172659_EUb8vUo.jpg  
  inflating: data/obj/110_VID20220624172659_EUb8vUo.txt  
  inflating: data/obj/114_VID20220624173023.jpg  
  inflating: data/obj/114_VID20220624173023.txt  
  inflating: data/obj/115_VID20220624173023.jpg  
  inflating: data/obj/115_VID20220624173023.txt  
  inflating: data/obj/118_VID20220624173023_hEIYLj2.jpg  
  inflating: data/obj/118_VID20220624173023_hEIYLj2.txt  
  i

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.504484 0.523438 0.246637 0.338542

['0', '0.504484', '0.523438', '0.246637', '0.338542']
0 data/obj/467_VID20220629163116.txt
0 0.504484 0.523438 0.246637 0.338542
0 0.430636 0.622449 0.190751 0.336735

['0', '0.430636', '0.622449', '0.190751', '0.336735']
1 data/obj/487_VID20220629164607.txt
0 0.430636 0.622449 0.190751 0.336735
0 0.623762 0.446970 0.138614 0.217172

['0', '0.623762', '0.446970', '0.138614', '0.217172']
2 data/obj/524_VID20220629170427.txt
0 0.623762 0.446970 0.138614 0.217172
0 0.283626 0.673228 0.204678 0.259843

['0', '0.283626', '0.673228', '0.204678', '0.259843']
3 data/obj/406_VID20220629105403.txt
0 0.283626 0.673228 0.204678 0.259843
0 0.766484 0.487500 0.104396 0.225000

['0', '0.766484', '0.487500', '0.104396', '0.225000']
4 data/obj/407_VID20220629105403.txt
0 0.766484 0.487500 0.104396 0.225000
0 0.506803 0.471545 0.238095 0.211382

['0', '0.506803', '0.471545', '0.238095', '0.211382']
5 data/obj/291_VID20220628175103.txt
0 0.506803 0.471545 0.238095 0

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/160_VID20220624174747_BLPgaDz.jpg', 'data/obj/387_VID20220629104127.jpg', 'data/obj/339_VID20220628190803.jpg', 'data/obj/559_VID20220629170757.jpg', 'data/obj/151_VID20220624174747.jpg', 'data/obj/829_VID_20220712_170708.jpg', 'data/obj/20_VID20220622153603.jpg', 'data/obj/464_VID20220629163116.jpg', 'data/obj/408_VID20220629105403.jpg', 'data/obj/574_VID20220629170954.jpg', 'data/obj/410_VID20220629105403.jpg', 'data/obj/612_VID20220629174207.jpg', 'data/obj/10_VID20220622145715.jpg', 'data/obj/654_VID20220706161602.jpg', 'data/obj/626_VID20220629174625_R6UD1XE.jpg', 'data/obj/288_VID20220628175014.jpg', 'data/obj/409_VID20220629105403.jpg', 'data/obj/77_VID20220624172155_Yz1wdGp.jpg', 'data/obj/187_VID20220626182013.jpg', 'data/obj/137_VID20220624174404.jpg', 'data/obj/657_VID20220706161745.jpg', 'data/obj/118_VID20220624173023_hEIYLj2.jpg', 'data/obj/70_VID20220624171723_9bTqCxH.jpg', 'data/obj/115_VID20220624173023.jpg', 'data/obj/682_VID20220706174505.jpg', 'data/obj/4

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.009623, iou_loss = 0.000000, total_loss = 0.009623 
 total_bbox = 183616, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.900392), count: 2, class_loss = 0.000883, iou_loss = 0.017667, total_loss = 0.018551 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.736827), count: 4, class_loss = 0.505001, iou_loss = 0.370178, total_loss = 0.875179 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000006, iou_loss = 0.000000, total_loss = 0.000006 
 total_bbox = 183622, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.855464), count: 3, class_loss = 0.159798, iou_loss = 0.044873, total_loss = 0.204671 
v3 (m